In [ ]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return len(t)*t/np.sum(t)

In [ ]:
def par(n,q0,Ec,delta):
    return Ec*(n-q0)**2+delta*(1-(-1)**n)/2
def find_oc(q0,Ec,delta):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=[e1,e2,e3]
    return a+b.index(min(b))-1

def oc(q0,Ec,delta,T):
    a=int(q0)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    b=np.array([e1,e2,e3])
    Z=sum(np.exp(-b/T))
    return np.exp(-b/T)/Z,a

def exc(q0,Ec,delta,T,deltat):
    b,a=oc(q0,Ec,delta,T)
    e0=par(a-2,q0,Ec,delta)
    e1=par(a-1,q0,Ec,delta)
    e2=par(a,q0,Ec,delta)
    e3=par(a+1,q0,Ec,delta)
    e4=par(a+1,q0,Ec,delta)
    return [b[0],b[0],b[1],b[1],b[2],b[2]],[np.abs(e2-e1)+deltat,-np.abs(e1-e0)-deltat,np.abs(e3-e2)+deltat,-np.abs(e1-e2)-deltat,np.abs(e4-e3)+deltat,-np.abs(e2-e3)-deltat]

fig,ax=plt.subplots(2)

Ec=1
delta=1
q0=1.2
T=0.1
deltat=1
N=range(-5,5)
x=np.linspace(-4,4,100)
for i in N:
    ax[0].plot(x,par(i,x,Ec,delta))

n=find_oc(q0,Ec,delta)
ax[0].scatter(q0,par(n,q0,Ec,delta))
ax[0].set_ylim(0,3)
a,b=exc(q0,Ec,delta,T,deltat)
z=np.zeros(len(x))
for i in range(len(a)):
    z+=a[i]/((x-b[i])**2+0.1)

ax[1].plot(x,z)


axfreq = fig.add_axes([0.25, 0.0, 0.65, 0.03])
q0s = Slider(
    ax=axfreq,
    label='Max',
    valmin=-3,
    valmax=3,
    valinit=0,
)

axfreq = fig.add_axes([0.25, 0.03, 0.65, 0.03])
Ts = Slider(
    ax=axfreq,
    label='Max',
    valmin=0,
    valmax=2,
    valinit=0.1,
)


axamp = fig.add_axes([0.03, 0.25, 0.0225, 0.63])
Ecs = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=2,
    valinit=1,
    orientation="vertical"
)

axamp = fig.add_axes([0.01, 0.25, 0.0225, 0.63])
etas = Slider(
    ax=axamp,
    label="Min",
    valmin=0,
    valmax=1,
    valinit=0.1,
    orientation="vertical"
)

def update(val):
    ax[0].clear()
    ax[1].clear()
    for i in N:
        ax[0].plot(x,par(i,x,Ecs.val,delta))

    n=find_oc(q0s.val,Ecs.val,delta)
    ax[0].scatter(q0s.val,par(n,q0s.val,Ecs.val,delta))
    ax[0].set_ylim(0,3)
    a,b=exc(q0s.val,Ecs.val,delta,Ts.val,deltat)
    z=np.zeros(len(x))
    for i in range(len(a)):
        z+=a[i]/((x-b[i])**2+etas.val)
    ax[1].plot(x,z)
    

q0s.on_changed(update)
Ecs.on_changed(update) 
Ts.on_changed(update)  
etas.on_changed(update)  
